In [1]:
import pandas as pd

import pytorch_lightning as pl
from pytorch_lightning import Trainer

from massspecgym.datasets import MassSpecDataModule
from massspecgym.preprocessors import SpecTokenizer, MolFingerprinter
from massspecgym.models.retrieval.deepsets import DeepSetsRetrieval
%load_ext autoreload
%autoreload 2

## Train and evaluate a simple Deep Sets model on the fingerprint retrieval task

In [2]:
# Load and split data
split_mask = pd.read_csv('../data/example_5_spectra_split.csv')['fold']
spec_preproc = SpecTokenizer(n_peaks=60)
mol_preproc = MolFingerprinter()
data_module = MassSpecDataModule(
    mgf_pth='../data/example_5_spectra.mgf',
    spec_preproc=spec_preproc,
    mol_preproc=mol_preproc,
    split_mask=split_mask,
    batch_size=2
)
data_module.prepare_data()
data_module.setup()

# Init model
model = DeepSetsRetrieval()

# Init logger
# You may need to run wandb init first to use the wandb logger
# Alternatively set logger = None not to use wandb
entity = None
project = 'deepsets_retrieval'
name = 'debug_run'
logger = pl.loggers.WandbLogger(
    entity=entity,
    project=project,
    name=name,
    tags=[],
    log_model=False,
)

# Init trainer
trainer = Trainer(
    accelerator='cpu',
    max_epochs=200,
    logger=logger,
    log_every_n_steps=1
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/anton/miniconda3/envs/massspecgym/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [3]:
# Train
trainer.fit(model, datamodule=data_module)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anton-bushuiev. Use `wandb login --relogin` to force relogin



  | Name | Type       | Params
------------------------------------
0 | phi  | Sequential | 16.9 K
1 | rho  | Sequential | 280 K 
------------------------------------
297 K     Trainable params
0         Non-trainable params
297 K     Total params
1.190     Total estimated model params size (MB)


/Users/anton/miniconda3/envs/massspecgym/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/anton/miniconda3/envs/massspecgym/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 199: 100%|██████████| 2/2 [00:00<00:00, 68.01it/s, v_num=um41, train_loss_step=0.383, val_loss_step=0.380] 

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 2/2 [00:00<00:00, 55.67it/s, v_num=um41, train_loss_step=0.383, val_loss_step=0.380]
